<a href="https://colab.research.google.com/github/Shreyarajpal12/Watermarking-summarized-text/blob/main/Watermarking_language_models_for_summarizing_CNN_mails_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Watermarking Language Models

#### Experiment description
The watermarking algorithm reliably identifies AI-generated summaries by embedding detectable patterns, evidenced by higher Z-scores and green fractions in watermarked text, ensuring robust detection while preserving quality. This experiment utilized sequence-to-sequence models pre-trained on summarization tasks, such as T5 and FLAN, with the CNN/Daily Mail dataset as the benchmark.

In [ ]:
!pip install transformers gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 109.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
!git clone https://github.com/jwkirchenbauer/lm-watermarking.git

Cloning into 'lm-watermarking'...
remote: Enumerating objects: 336, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 336 (delta 20), reused 10 (delta 6), pack-reused 289 (from 1)
Receiving objects: 100% (336/336), 12.00 MiB | 16.60 MiB/s, done.
Resolving deltas: 100% (99/99), done.


In [ ]:
## for easier comprehensibility the cloned github is not uploaded directly in this repo .

In [ ]:
%cd lm-watermarking

/content/lm-watermarking/lm-watermarking


#### Experiment 1: Using palmyra model

In [ ]:
from extended_watermark_processor import WatermarkLogitsProcessor
from transformers import AutoTokenizer, AutoModelForCausalLM, LogitsProcessorList
import torch

# Initialize model and tokenizer
model_name = "Writer/palmyra-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).cuda()

# Initialize the watermark processor with a valid seeding scheme
watermark_processor = WatermarkLogitsProcessor(
    vocab=list(tokenizer.get_vocab().values()),
    gamma=0.25,  # Fraction of tokens in the greenlist
    delta=2.0,   # Bias for greenlist tokens
    seeding_scheme="selfhash"  # Ensure this is correctly set
)

# Input text for watermarking
input_text = "This is a sample input text to generate watermarked output."

# Tokenize input
tokenized_input = tokenizer(input_text, return_tensors="pt").to(model.device)

# Check input length matches requirements for the context width
if tokenized_input["input_ids"].shape[-1] < 4:
    raise ValueError("Input text must have at least 4 tokens to meet the seeding scheme's requirements.")

# Generate watermarked text
output_tokens = model.generate(
    **tokenized_input,
    logits_processor=LogitsProcessorList([watermark_processor]),
    max_length=50
)

# Decode the watermarked output
watermarked_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
print("Watermarked Output:", watermarked_text)




Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Watermarked Output: This is a sample input text to generate watermarked output.
                                     


#### Experiment 1: Using OPT Small language model

In [ ]:
from extended_watermark_processor import WatermarkLogitsProcessor
from transformers import AutoTokenizer, AutoModelForCausalLM, LogitsProcessorList

# Load Hugging Face model and tokenizer
model_name = "facebook/opt-1.3b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).cuda()

# Initialize WatermarkLogitsProcessor
watermark_processor = WatermarkLogitsProcessor(
    vocab=list(tokenizer.get_vocab().values()),
    gamma=0.25,  # Fraction of greenlist tokens
    delta=2.0,   # Strength of watermarking
    seeding_scheme="selfhash"  # Seeding scheme; alternatives: "minhash", etc.
)

# Input text
input_text = "Government regulation of artificial intelligence is essential to ensure safety and fairness. As AI technologies become more integrated into society, proper oversight can prevent misuse and protect civil liberties. However, regulation must be balanced to encourage innovation and growth in the AI sector."

# Tokenize input
tokenized_input = tokenizer(input_text, return_tensors="pt").to(model.device)

# Generate watermarked text
output_tokens = model.generate(
    **tokenized_input,
    logits_processor=LogitsProcessorList([watermark_processor]),
    max_length=250
)

# Remove input prompt (if decoder-only model)
output_tokens = output_tokens[:, tokenized_input["input_ids"].shape[-1]:]

# Decode generated text
watermarked_text = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)[0]
print("Watermarked Output:", watermarked_text)


Watermarked Output: 

A recent survey by the Pew Research Center found that the majority of Americans, including those who oppose regulation, are concerned about AI’s ability to affect human rights. The same survey found that the majority of Americans, including those who oppose regulation, are concerned about AI’s ability to affect human rights.

A recent survey by the Pew Research Center found that the majority of Americans, including those who oppose regulation, are concerned about AI’s ability to affect human rights. The same survey found that the majority of Americans, including those who oppose regulation, are concerned about AI’s ability to affect human rights.

A recent survey by the Pew Research Center found that the majority of Americans, including those who oppose regulation, are concerned about AI’s ability to affect human rights. The same survey found that the majority of Americans, including those who oppose regulation, are concerned about AI’s ability to affect human rig

In [ ]:
from extended_watermark_processor import WatermarkLogitsProcessor
from transformers import AutoTokenizer, AutoModelForCausalLM, LogitsProcessorList
import torch
# Load Hugging Face model and tokenizer
model_name = "facebook/opt-1.3b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).cuda()

# Initialize WatermarkLogitsProcessor
watermark_processor = WatermarkLogitsProcessor(
    vocab=list(tokenizer.get_vocab().values()),
    gamma=0.25,
    delta=2.0,
    seeding_scheme="selfhash"  # Ensure this matches your context requirements
)

# Input text
input_text = "summarize this: Government regulation of artificial intelligence is essential to ensure safety and fairness. As AI technologies become more integrated into society, proper oversight can prevent misuse and protect civil liberties. However, regulation must be balanced to encourage innovation and growth in the AI sector"

# Tokenize input
tokenized_input = tokenizer(input_text, return_tensors="pt").to(model.device)

# Ensure input meets minimum length for seeding
min_length = 4
if tokenized_input["input_ids"].shape[-1] < min_length:
    padding_tokens = tokenizer.encode("padding ", return_tensors="pt").to(model.device)
    tokenized_input["input_ids"] = torch.cat([padding_tokens, tokenized_input["input_ids"]], dim=-1)

# Generate watermarked text
output_tokens = model.generate(
    **tokenized_input,
    logits_processor=LogitsProcessorList([watermark_processor]),
    max_length=250
)

# Decode generated text
watermarked_text = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)[0]
print("Watermarked Output:", watermarked_text)


Watermarked Output: summarize this: Government regulation of artificial intelligence is essential to ensure safety and fairness. As AI technologies become more integrated into society, proper oversight can prevent misuse and protect civil liberties. However, regulation must be balanced to encourage innovation and growth in the AI sector while ensuring safety and fairness for the public.

A recent article by the Brookings Institution argues that the government’s approach to regulating AI needs to be more nuanced and more effective. In order to accomplish this, the Brookings authors suggest the following principles for the federal government’s approach to regulation of AI:

A broad, enforceable set of standards and best practices for AI technologies

A system of checks and balances to ensure that the government’s approach to regulation of AI technologies is appropriate and effective

A system of accountability, transparency, and transparency

A system of accountability, transparency, and

#### Checking to see how the scores and responses change when watermarking is used and when it isnt, using Gemma 2b model

In [ ]:
from extended_watermark_processor import WatermarkLogitsProcessor
from transformers import AutoTokenizer, AutoModelForCausalLM, LogitsProcessorList
import torch

# Load Seq2Seq model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b-it").cuda()

# Initialize WatermarkLogitsProcessor
watermark_processor = WatermarkLogitsProcessor(
    vocab=list(tokenizer.get_vocab().values()),
    gamma=0.25,
    delta=2.0,
    seeding_scheme="selfhash"  # Ensure this matches your context requirements
)

# Input text
input_text = "summarize this: Government regulation of artificial intelligence is essential to ensure safety and fairness. As AI technologies become more integrated into society, proper oversight can prevent misuse and protect civil liberties. However, regulation must be balanced to encourage innovation and growth in the AI sector"

# Tokenize input
tokenized_input = tokenizer(input_text, return_tensors="pt").to(model.device)

# Ensure input meets minimum length for seeding
min_length = 4
if tokenized_input["input_ids"].shape[-1] < min_length:
    padding_tokens = tokenizer.encode("padding ", return_tensors="pt").to(model.device)
    tokenized_input["input_ids"] = torch.cat([padding_tokens, tokenized_input["input_ids"]], dim=-1)

# Generate watermarked text
output_tokens = model.generate(
    **tokenized_input,
    logits_processor=LogitsProcessorList([watermark_processor]),
    max_length=250
)

# Decode generated text
watermarked_text = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)[0]
print("Watermarked Output:", watermarked_text)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Watermarked Output: summarize this: Government regulation of artificial intelligence is essential to ensure safety and fairness. As AI technologies become more integrated into society, proper oversight can prevent misuse and protect civil liberties. However, regulation must be balanced to encourage innovation and growth in the AI sector. This is a complex and evolving issue with no easy solutions.

Here's a breakdown of the key points:

**Why regulation is necessary:**

* **Safety:** AI systems can pose risks, such as malfunctions and biases in decision-making.
* **Fairness:** AI can perpetuate and amplify existing societal biases if not carefully designed and monitored.
* **Privacy:** AI can collect and analyze vast amounts of personal data, raising privacy concerns.
* **Transparency:** The decision-making processes of complex AI systems can be opaque, making it difficult to understand and trust their outputs.

**Challenges and solutions:**

* **Defining clear boundaries:** Establishi

In [ ]:
watermarked_text

"summarize this: Government regulation of artificial intelligence is essential to ensure safety and fairness. As AI technologies become more integrated into society, proper oversight can prevent misuse and protect civil liberties. However, regulation must be balanced to encourage innovation and growth in the AI sector. This is a complex and evolving issue with no easy solutions.\n\nHere's a breakdown of the key points:\n\n**Why regulation is necessary:**\n\n* **Safety:** AI systems can pose risks, such as malfunctions and biases in decision-making.\n* **Fairness:** AI can perpetuate and amplify existing societal biases if not carefully designed and monitored.\n* **Privacy:** AI can collect and analyze vast amounts of personal data, raising privacy concerns.\n* **Transparency:** The decision-making processes of complex AI systems can be opaque, making it difficult to understand and trust their outputs.\n\n**Challenges and solutions:**\n\n* **Defining clear boundaries:** Establishing cle

##### Scores of Text without watermark

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b-it").cuda()

# Input text
input_text = "summarize this: Government regulation of artificial intelligence is essential to ensure safety and fairness. As AI technologies become more integrated into society, proper oversight can prevent misuse and protect civil liberties. However, regulation must be balanced to encourage innovation and growth in the AI sector"

# Tokenize the input text
inputs = tokenizer(input_text, return_tensors="pt").to("cuda")

# Generate output (using greedy decoding as an example)
output_tokens = model.generate(
    inputs.input_ids,
    max_length=250,  # Adjust max length as needed
    num_beams=5,    # Number of beams for beam search (optional, default is 1)
    no_repeat_ngram_size=2,  # Avoid repetition
    early_stopping=True      # Stop early when beams converge
)

# Decode the output tokens back to text
output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

# Print the output
print("Generated Output:", output_text)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generated Output: summarize this: Government regulation of artificial intelligence is essential to ensure safety and fairness. As AI technologies become more integrated into society, proper oversight can prevent misuse and protect civil liberties. However, regulation must be balanced to encourage innovation and growth in the AI sector.

Here's a breakdown of the arguments:

**For regulation:**

* **Safety:** AI systems can pose significant safety risks, especially in areas like autonomous vehicles and healthcare. Regulations can mitigate these risks by setting standards for safety testing, data security, and algorithmic transparency.
*  **Fairness:**  AI algorithms can perpetuate existing societal biases, leading to discriminatory outcomes. Regulation can address this by promoting fairness in AI development and deployment, ensuring algorithms are trained on diverse data sets and are subject to regular audits for bias. 
    
**Against regulation (or at least limited regulation):**
 * **

In [ ]:
output_text

"summarize this: Government regulation of artificial intelligence is essential to ensure safety and fairness. As AI technologies become more integrated into society, proper oversight can prevent misuse and protect civil liberties. However, regulation must be balanced to encourage innovation and growth in the AI sector.\n\nHere's a breakdown of the arguments:\n\n**For regulation:**\n\n* **Safety:** AI systems can pose significant safety risks, especially in areas like autonomous vehicles and healthcare. Regulations can mitigate these risks by setting standards for safety testing, data security, and algorithmic transparency.\n*  **Fairness:**  AI algorithms can perpetuate existing societal biases, leading to discriminatory outcomes. Regulation can address this by promoting fairness in AI development and deployment, ensuring algorithms are trained on diverse data sets and are subject to regular audits for bias. \n    \n**Against regulation (or at least limited regulation):**\n * **Stiflin

In [ ]:
watermark_detector = WatermarkDetector(vocab=list(tokenizer.get_vocab().values()),
                                        gamma=0.25, # should match original setting
                                        seeding_scheme="selfhash", # should match original setting
                                        device=model.device, # must match the original rng device type
                                        tokenizer=tokenizer,
                                        z_threshold=4.0,
                                        normalizers=[],
                                        ignore_repeated_ngrams=True)

score_dict = watermark_detector.detect(output_text) # can be replaced with watermark_text to see the difference

In [ ]:
score_dict # for plain text , the green fraction here is 0.25 but in watermarked text is around 0.32 drawing a clear distinction between watermarked and plain text.

{'num_tokens_scored': 246,
 'num_green_tokens': 62,
 'green_fraction': 0.25203252032520324,
 'z_score': 0.07362101738323103,
 'p_value': 0.4706559735331334,
 'z_score_at_T': tensor([-0.5774, -0.8165, -1.0000, -1.1547, -1.2910, -1.4142, -1.5275, -1.6330,
         -1.7321, -1.8257, -1.2185, -0.6667, -0.8006, -0.3086, -0.4472, -0.5774,
         -0.7001, -0.2722, -0.3974, -0.5164, -0.6299, -0.2462, -0.3612, -0.4714,
         -0.5774, -0.2265, -0.3333, -0.4364, -0.1072, -0.2108, -0.3111, -0.4082,
         -0.5025, -0.5941, -0.6831, -0.7698, -0.8542, -0.9366, -1.0170, -1.0954,
         -1.1722, -1.2472, -1.3207, -1.3926, -1.4631, -1.5323, -1.6001, -1.6667,
         -1.7321, -1.7963, -1.8594, -1.9215, -1.9826, -1.7285, -1.7905, -1.8516,
         -1.9118, -1.9711, -1.7288, -1.7889, -1.5524, -1.3198, -1.3820, -1.4434,
         -1.5038, -1.5635, -1.3402, -1.1202, -0.9036, -0.9661, -1.0278, -0.8165,
         -0.8785, -0.9396, -0.7333, -0.5298, -0.5922, -0.6537, -0.7145, -0.7746,
         -0.8340,

### Differentiating in the watermarked scores between Causal and Sequence-2-Sequence models -> opt vs t5 and flan

In [ ]:
# Required imports
import torch
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, LogitsProcessorList
from extended_watermark_processor import WatermarkLogitsProcessor, WatermarkDetector
# from extended_watermark_processor import WatermarkLogitsProcessor
from transformers import AutoTokenizer, AutoModelForCausalLM, LogitsProcessorList
# import torch
# Load Hugging Face model and tokenizer
model_name = "facebook/opt-1.3b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).cuda()
# # Load the FLAN model and tokenizer
# model_name = "google/flan-t5-large"  # You can replace this with another FLAN variant
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the watermark logits processor
watermark_processor = WatermarkLogitsProcessor(
    vocab=list(tokenizer.get_vocab().values()),
    gamma=0.25,
    delta=2.0,
    seeding_scheme="selfhash"  # Equivalent to `ff-anchored_minhash_prf-4-True-15485863`
)

# Input text for processing
input_text = """The quick brown fox jumps over the lazy dog. The fox is known for its agility and cunning nature,
while the dog is often seen as a loyal and faithful companion. """

# Tokenize the input
tokenized_input = tokenizer(input_text, return_tensors="pt").to(model.device)
print(tokenized_input)

# Generate output with watermarking applied
output_tokens = model.generate(
    **tokenized_input,
    logits_processor=LogitsProcessorList([watermark_processor]),
    max_length=50

)

# If the model is decoder-only, isolate the generated tokens
output_tokens = output_tokens[:, tokenized_input["input_ids"].shape[-1]:]

# Decode the generated output
output_text = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)[0]
print("Generated Output with Watermark:", output_text)

watermark_detector = WatermarkDetector(vocab=list(tokenizer.get_vocab().values()),
                                        gamma=0.25, # should match original setting
                                        seeding_scheme="selfhash", # should match original setting
                                        device=model.device, # must match the original rng device type
                                        tokenizer=tokenizer,
                                        z_threshold=4.0,
                                        normalizers=[],
                                        ignore_repeated_ngrams=True)

score_dict = watermark_detector.detect(output_text) # or any other text of interest to analyze

# Detect watermark in the output
# score_dict = watermark_detector.detect(output_text)

# Print watermark detection scores
print("Watermark Detection Scores:", score_dict)

# Example analysis of watermark detection
if score_dict['z_score'] > watermark_detector.z_threshold:
    print("The text is likely watermarked.")
else:
    print("The text is not watermarked.")


{'input_ids': tensor([[    2,   133,  2119,  6219, 23602, 13855,    81,     5, 22414,  2335,
             4,    20, 23602,    16,   684,    13,    63, 28924,     8, 41526,
          2574,     6,  1437, 50118, 20235,     5,  2335,    16,   747,   450,
            25,    10,  8259,     8, 15828, 15625,     4,  1437]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
Generated Output with Watermark:  I think I've been conditioned by this.

I
Watermark Detection Scores: {'num_tokens_scored': 9, 'num_green_tokens': 8, 'green_fraction': 0.8888888888888888, 'z_score': 4.426352063787131, 'p_value': 4.792002833235625e-06, 'z_score_at_T': tensor([1.7321, 2.4495, 3.0000, 3.4641, 3.8730, 4.2426, 4.5826, 4.8990, 4.4264]), 'prediction': True, 'confidence': 0.9999952079971668}
The text is likely watermarked.


#### Using Flan-T5 and T5 small to get watermarked vs non-watermarked scores and summaries on CNN mail dataset.

In [ ]:
# Required imports
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, LogitsProcessorList
from extended_watermark_processor import WatermarkLogitsProcessor, WatermarkDetector

# Load the Seq2Seq model (FLAN-T5) and tokenizer
model_name = "google/flan-t5-large"  # Replace with your desired Seq2Seq model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).cuda()

# Initialize the watermark logits processor
watermark_processor = WatermarkLogitsProcessor(
    vocab=list(tokenizer.get_vocab().values()),
    gamma=0.25,
    delta=2.0,
    seeding_scheme="simple_1"
)

# Input text for processing
input_text = """The quick brown fox jumps over the lazy dog. The fox is known for its agility and cunning nature,
while the dog is often seen as a loyal and faithful companion."""

# Tokenize the input
tokenized_input = tokenizer(input_text, return_tensors="pt").to(model.device)

# Check for valid tokenized input
if tokenized_input["input_ids"].size(1) < 4:
    raise ValueError("Input text is too short for the watermarking seeding scheme. Please provide longer text.")

# Generate output with watermarking applied
output_tokens = model.generate(
    **tokenized_input,
    logits_processor=LogitsProcessorList([watermark_processor]),
    max_length=50,  # Ensure sufficient length for meaningful output
    num_beams=5  # Use beam search for better outputs
)

# Validate generated output tokens
if output_tokens is None or output_tokens.size(0) == 0:
    raise ValueError("Model generated no tokens. Adjust input or generation parameters.")

# Decode the generated output (no slicing needed for Seq2Seq models)
output_text = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)[0]

# Ensure decoded text is non-empty
if not output_text.strip():
    raise ValueError("Decoded text is empty. Please verify the generation process.")

# Print the generated output
print("Generated Output with Watermark:", output_text)

# Initialize the watermark detector
watermark_detector = WatermarkDetector(
    vocab=list(tokenizer.get_vocab().values()),
    gamma=0.25,  # Match the original watermark settings
    seeding_scheme="simple_1",  # Match the original seeding scheme
    device=model.device,  # Must match the original RNG device type
    tokenizer=tokenizer,
    z_threshold=1.0,  # Adjust as needed
    normalizers=[],  # Use any additional normalizers if necessary
    ignore_repeated_ngrams=True  # Avoid false positives from repeated n-grams
)

# Detect watermark in the output
try:
    score_dict = watermark_detector.detect(output_text)
    print("Watermark Detection Scores:", score_dict)

    # Analyze watermark detection
    if score_dict['z_score'] > watermark_detector.z_threshold:
        print("The text is likely watermarked.")
    else:
        print("The text is not watermarked.")
except IndexError as e:
    print(f"Error in watermark detection: {e}")
    print("Ensure the output text is properly formatted and non-empty.")


Generated Output with Watermark: foxes can be very dangerous
Watermark Detection Scores: {'num_tokens_scored': 7, 'num_green_tokens': 5, 'green_fraction': 0.7142857142857143, 'z_score': 2.836832573067901, 'p_value': 0.0022781749015925654, 'z_score_at_T': tensor([-0.5774,  0.8165,  0.3333,  1.1547,  1.8074,  2.3570,  2.8368]), 'prediction': True, 'confidence': 0.9977218250984075}
The text is likely watermarked.


#### implementing this on the 60+ rows in the dataset

In [ ]:
#### pardon my usage of keyboard interrupt i printed these as results and then saved in a file.

In [ ]:
# Required imports
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, LogitsProcessorList
from extended_watermark_processor import WatermarkLogitsProcessor, WatermarkDetector
from datasets import load_dataset
import json

# Load the CNN/Daily Mail dataset
dataset = load_dataset("cnn_dailymail", "3.0.0", split="test")  # Use 'test' split for inference

# Load the T5-Small model and tokenizer
model_name = "t5-small"  # Use T5-Small for smaller and faster processing
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).cuda()

# Initialize the watermark logits processor
watermark_processor = WatermarkLogitsProcessor(
    vocab=list(tokenizer.get_vocab().values()),
    gamma=0.25,
    delta=2.0,
    seeding_scheme="simple_1"  # Adjust as needed
)

# Initialize the watermark detector
watermark_detector = WatermarkDetector(
    vocab=list(tokenizer.get_vocab().values()),
    gamma=0.25,  # Match the original watermark settings
    seeding_scheme="simple_1",  # Match the original seeding scheme
    device=model.device,  # Must match the original RNG device type
    tokenizer=tokenizer,
    z_threshold=4.0,  # Adjust as needed
    normalizers=[],  # Use any additional normalizers if necessary
    ignore_repeated_ngrams=True  # Avoid false positives from repeated n-grams
)

# Function to summarize text with or without watermarking
def summarize_and_detect_scores(
    text, model, tokenizer, watermark_processor, watermark_detector, apply_watermark, max_length=150
):
    # Tokenize the input
    tokenized_input = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(model.device)

    # Check for valid tokenized input
    if tokenized_input["input_ids"].size(1) < 4:
        raise ValueError("Input text is too short for the watermarking seeding scheme. Please provide longer text.")

    # Generate summary
    if apply_watermark:
        # With watermarking
        output_tokens = model.generate(
            **tokenized_input,
            logits_processor=LogitsProcessorList([watermark_processor]),
            max_length=max_length,
            num_beams=5
        )
    else:
        # Without watermarking
        output_tokens = model.generate(
            **tokenized_input,
            max_length=max_length,
            num_beams=5
        )

    # Validate generated output tokens
    if output_tokens is None or output_tokens.size(0) == 0:
        raise ValueError("Model generated no tokens. Adjust input or generation parameters.")

    # Decode the generated summary
    output_text = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)[0]

    # Ensure decoded text is non-empty
    if not output_text.strip():
        raise ValueError("Decoded text is empty. Please verify the generation process.")

    # Detect watermark in the output
    score_dict = watermark_detector.detect(output_text)

    return output_text, score_dict

# Process the CNN/Daily Mail dataset
results = []
for i, example in enumerate(dataset):
    try:
        # Input article
        article = example["article"]

        # Summarize with watermark
        summary_with_watermark, scores_with_watermark = summarize_and_detect_scores(
            article, model, tokenizer, watermark_processor, watermark_detector, apply_watermark=True
        )

        # Summarize without watermark
        summary_without_watermark, scores_without_watermark = summarize_and_detect_scores(
            article, model, tokenizer, watermark_processor, watermark_detector, apply_watermark=False
        )

        # Save the results
        results.append({
            "article": article,
            "summary_with_watermark": summary_with_watermark,
            "watermark_scores_with_watermark": scores_with_watermark,
            "summary_without_watermark": summary_without_watermark,
            "watermark_scores_without_watermark": scores_without_watermark
        })

        # Print progress every 10 examples
        if i % 10 == 0:
            print(f"Processed {i} examples.")
            print(f"Sample Summary With Watermark:\n{summary_with_watermark}")
            print(f"Sample Watermark Scores With Watermark:\n{scores_with_watermark}")
            print(f"Sample Summary Without Watermark:\n{summary_without_watermark}")
            print(f"Sample Watermark Scores Without Watermark:\n{scores_without_watermark}\n")
    except Exception as e:
        print(f"Error processing example {i}: {e}")
        continue

# Save summaries and scores to a file
with open("cnn_dailymail_summaries_with_and_without_watermark_t5_small.json", "w") as f:
    json.dump(results, f, indent=2)

print("Summarization complete. Results saved to 'cnn_dailymail_summaries_with_and_without_watermark_t5_small.json'.")


Processed 0 examples.
Sample Summary With Watermark:
is the 123rd member of the International Criminal Court on Wednesday. the Palestinian Authority officially became the 123rd member of the international criminal court. the court gives the court jurisdiction over alleged crimes in Palestinian territories. the ICC opened a preliminary examination into the situation in Palestinian territories.
Sample Watermark Scores With Watermark:
{'num_tokens_scored': 46, 'num_green_tokens': 19, 'green_fraction': 0.41304347826086957, 'z_score': 2.553769592276246, 'p_value': 0.005328187360645716, 'z_score_at_T': tensor([1.7321, 0.8165, 1.6667, 1.1547, 0.7746, 0.4714, 1.0911, 1.6330, 1.3472,
        1.8257, 1.5667, 1.3333, 1.1209, 0.9258, 1.3416, 1.7321, 1.5403, 1.3608,
        1.1921, 1.0328, 1.0328, 1.0328, 1.0328, 1.0328, 1.0328, 1.0328, 1.0328,
        0.8819, 0.7385, 0.6019, 0.4714, 0.4714, 0.8083, 1.1323, 1.4444, 1.4444,
        1.3093, 1.1793, 1.0541, 1.3480, 1.6330, 1.5076, 1.7823, 2.0494, 2.30

KeyboardInterrupt: 

In [ ]:
# Required imports
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, LogitsProcessorList
from extended_watermark_processor import WatermarkLogitsProcessor, WatermarkDetector
from datasets import load_dataset
import json

# Load the CNN/Daily Mail dataset
dataset = load_dataset("cnn_dailymail", "3.0.0", split="test")  # Use 'test' split for inference

# Load the Seq2Seq model (FLAN-T5) and tokenizer
model_name = "google/flan-t5-large"  # Replace with your desired Seq2Seq model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).cuda()

# Initialize the watermark logits processor
watermark_processor = WatermarkLogitsProcessor(
    vocab=list(tokenizer.get_vocab().values()),
    gamma=0.25,
    delta=2.0,
    seeding_scheme="simple_1"  # Adjust as needed
)

# Initialize the watermark detector
watermark_detector = WatermarkDetector(
    vocab=list(tokenizer.get_vocab().values()),
    gamma=0.25,  # Match the original watermark settings
    seeding_scheme="simple_1",  # Match the original seeding scheme
    device=model.device,  # Must match the original RNG device type
    tokenizer=tokenizer,
    z_threshold=4.0,  # Adjust as needed
    normalizers=[],  # Use any additional normalizers if necessary
    ignore_repeated_ngrams=True  # Avoid false positives from repeated n-grams
)

# Function to summarize text with or without watermarking
def summarize_and_detect_scores(
    text, model, tokenizer, watermark_processor, watermark_detector, apply_watermark, max_length=150
):
    # Tokenize the input
    tokenized_input = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(model.device)

    # Check for valid tokenized input
    if tokenized_input["input_ids"].size(1) < 4:
        raise ValueError("Input text is too short for the watermarking seeding scheme. Please provide longer text.")

    # Generate summary
    if apply_watermark:
        # With watermarking
        output_tokens = model.generate(
            **tokenized_input,
            logits_processor=LogitsProcessorList([watermark_processor]),
            max_length=max_length,
            num_beams=5
        )
    else:
        # Without watermarking
        output_tokens = model.generate(
            **tokenized_input,
            max_length=max_length,
            num_beams=5
        )

    # Validate generated output tokens
    if output_tokens is None or output_tokens.size(0) == 0:
        raise ValueError("Model generated no tokens. Adjust input or generation parameters.")

    # Decode the generated summary
    output_text = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)[0]

    # Ensure decoded text is non-empty
    if not output_text.strip():
        raise ValueError("Decoded text is empty. Please verify the generation process.")

    # Detect watermark in the output
    score_dict = watermark_detector.detect(output_text)

    return output_text, score_dict

# Process the CNN/Daily Mail dataset
results = []
for i, example in enumerate(dataset):
    try:
        # Input article
        article = example["article"]

        # Summarize with watermark
        summary_with_watermark, scores_with_watermark = summarize_and_detect_scores(
            article, model, tokenizer, watermark_processor, watermark_detector, apply_watermark=True
        )

        # Summarize without watermark
        summary_without_watermark, scores_without_watermark = summarize_and_detect_scores(
            article, model, tokenizer, watermark_processor, watermark_detector, apply_watermark=False
        )

        # Save the results
        results.append({
            "article": article,
            "summary_with_watermark": summary_with_watermark,
            "watermark_scores_with_watermark": scores_with_watermark,
            "summary_without_watermark": summary_without_watermark,
            "watermark_scores_without_watermark": scores_without_watermark
        })

        # Print progress every 10 examples
        if i % 10 == 0:
            print(f"Processed {i} examples.")
            print(f"Sample Summary With Watermark:\n{summary_with_watermark}")
            print(f"Sample Watermark Scores With Watermark:\n{scores_with_watermark}")
            print(f"Sample Summary Without Watermark:\n{summary_without_watermark}")
            print(f"Sample Watermark Scores Without Watermark:\n{scores_without_watermark}\n")
    except Exception as e:
        print(f"Error processing example {i}: {e}")
        continue

# Save summaries and scores to a file
with open("cnn_dailymail_summaries_with_and_without_watermark.json", "w") as f:
    json.dump(results, f, indent=2)

print("Summarization complete. Results saved to 'cnn_dailymail_summaries_with_and_without_watermark.json'.")


Processed 0 examples.
Sample Summary With Watermark:
Palestine may be subject to counter-charges as well.
Sample Watermark Scores With Watermark:
{'num_tokens_scored': 11, 'num_green_tokens': 6, 'green_fraction': 0.5454545454545454, 'z_score': 2.263009527424072, 'p_value': 0.01181755099141695, 'z_score_at_T': tensor([1.7321, 2.4495, 1.6667, 1.1547, 1.8074, 1.4142, 1.0911, 0.8165, 1.3472,
        1.8257, 2.2630]), 'prediction': False}
Sample Summary Without Watermark:
Riad al-Malki said the move was a "step closer to ending a long era of impunity and injustice" and a "step closer to ending a long era of impunity and injustice."
Sample Watermark Scores Without Watermark:
{'num_tokens_scored': 31, 'num_green_tokens': 8, 'green_fraction': 0.25806451612903225, 'z_score': 0.10369516947304253, 'p_value': 0.4587056300960205, 'z_score_at_T': tensor([ 1.7321,  0.8165,  0.3333,  0.0000, -0.2582, -0.4714, -0.6547, -0.8165,
        -0.9623, -1.0954, -0.5222, -0.6667, -0.8006, -0.9258, -0.4472,  0.0

KeyboardInterrupt: 

In [ ]:
# # Required imports
# import torch
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, LogitsProcessorList
# from extended_watermark_processor import WatermarkLogitsProcessor, WatermarkDetector
# from datasets import load_dataset
# import json

# # Load the CNN/Daily Mail dataset
# dataset = load_dataset("cnn_dailymail", "3.0.0", split="test")  # Use 'test' split for inference

# # Load the BART model and tokenizer
# model_name = "facebook/bart-large-cnn"  # BART model specialized for summarization
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name).cuda()

# # Initialize the watermark logits processor
# watermark_processor = WatermarkLogitsProcessor(
#     vocab=list(tokenizer.get_vocab().values()),
#     gamma=0.25,
#     delta=2.0,
#     seeding_scheme="simple_1"  # Adjust as needed
# )

# # Initialize the watermark detector
# watermark_detector = WatermarkDetector(
#     vocab=list(tokenizer.get_vocab().values()),
#     gamma=0.25,  # Match the original watermark settings
#     seeding_scheme="simple_1",  # Match the original seeding scheme
#     device=model.device,  # Must match the original RNG device type
#     tokenizer=tokenizer,
#     z_threshold=4.0,  # Adjust as needed
#     normalizers=[],  # Use any additional normalizers if necessary
#     ignore_repeated_ngrams=True  # Avoid false positives from repeated n-grams
# )

# # Function to summarize text with or without watermarking
# def summarize_and_detect_scores(
#     text, model, tokenizer, watermark_processor, watermark_detector, apply_watermark, max_length=150
# ):
#     # Tokenize the input
#     tokenized_input = tokenizer(text, return_tensors="pt", truncation=True, max_length=1024).to(model.device)

#     # Check for valid tokenized input
#     if tokenized_input["input_ids"].size(1) < 4:
#         raise ValueError("Input text is too short for the watermarking seeding scheme. Please provide longer text.")

#     # Generate summary
#     if apply_watermark:
#         # With watermarking
#         output_tokens = model.generate(
#             **tokenized_input,
#             logits_processor=LogitsProcessorList([watermark_processor]),
#             max_length=max_length,
#             num_beams=5
#         )
#     else:
#         # Without watermarking
#         output_tokens = model.generate(
#             **tokenized_input,
#             max_length=max_length,
#             num_beams=5
#         )

#     # Validate generated output tokens
#     if output_tokens is None or output_tokens.size(0) == 0:
#         raise ValueError("Model generated no tokens. Adjust input or generation parameters.")

#     # Decode the generated summary
#     output_text = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)[0]

#     # Ensure decoded text is non-empty
#     if not output_text.strip():
#         raise ValueError("Decoded text is empty. Please verify the generation process.")

#     # Detect watermark in the output
#     score_dict = watermark_detector.detect(output_text)

#     return output_text, score_dict

# # Process the CNN/Daily Mail dataset
# results = []
# for i, example in enumerate(dataset):
#     try:
#         # Input article
#         article = example["article"]

#         # Summarize with watermark
#         summary_with_watermark, scores_with_watermark = summarize_and_detect_scores(
#             article, model, tokenizer, watermark_processor, watermark_detector, apply_watermark=True
#         )

#         # Summarize without watermark
#         summary_without_watermark, scores_without_watermark = summarize_and_detect_scores(
#             article, model, tokenizer, watermark_processor, watermark_detector, apply_watermark=False
#         )

#         # Save the results
#         results.append({
#             "article": article,
#             "summary_with_watermark": summary_with_watermark,
#             "watermark_scores_with_watermark": scores_with_watermark,
#             "summary_without_watermark": summary_without_watermark,
#             "watermark_scores_without_watermark": scores_without_watermark
#         })

#         # Print progress every 10 examples
#         if i % 10 == 0:
#             print(f"Processed {i} examples.")
#             print(f"Sample Summary With Watermark:\n{summary_with_watermark}")
#             print(f"Sample Watermark Scores With Watermark:\n{scores_with_watermark}")
#             print(f"Sample Summary Without Watermark:\n{summary_without_watermark}")
#             print(f"Sample Watermark Scores Without Watermark:\n{scores_without_watermark}\n")
#     except Exception as e:
#         print(f"Error processing example {i}: {e}")
#         continue

# # Save summaries and scores to a file
# with open("cnn_dailymail_summaries_with_and_without_watermark_bart.json", "w") as f:
#     json.dump(results, f, indent=2)

# print("Summarization complete. Results saved to 'cnn_dailymail_summaries_with_and_without_watermark_bart.json'.")


### results with T5 on cnn database

In [ ]:
results

[{'article': '(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC\'s founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians\' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesda

In [ ]:
import json
with open("cnn_dailymail_summaries.json", "w") as f:
    json.dump(results, f, indent=2)

#### These results are there in the results_{x}.txt, x being the model name. in the directory

#### Experimenting with t5 without watermarking on cnn dataset

In [ ]:
# Install necessary libraries
!pip install transformers datasets

from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import load_dataset

# Load the pre-trained T5 model and tokenizer
model_name = "t5-small"  # You can use other models like t5-base, t5-large, or t5-3b
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Load the CNN/Daily Mail dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")  # You can replace with 'xsum' or others

# Define a function to summarize a text
def summarize(text):
    # Preprocess the input text for T5
    inputs = tokenizer("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)

    # Generate the summary
    summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)

    # Decode the summary and return it
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Test summarization on a few examples from the CNN/Daily Mail dataset
for i in range(3):  # Summarize 3 samples from the dataset
    article = dataset['train'][i]['article']
    summary = summarize(article)
    print(f"Article {i+1}:")
    print(f"Original: {article[:300]}...")  # Display the first 300 characters
    print(f"Summary: {summary}")
    print("\n" + "-"*50 + "\n")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Article 1:
Original: LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappoi...
Summary: the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. he will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II" despite his growing fame, he says he is keeping his feet firmly on the ground.

--------------------------------------------------

Article 2:
Original: Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many men...
Sum